# Code Sandbox

A place to test new code and functions

In [1]:
# import packages and set working directory
import numpy as np
import matplotlib.pyplot as plt
import os
from math import exp
#os.chdir('../maize-Toff')

# import objects
from farm import Climate
from farm import Soil
from farm import Crop
from farm import CropModel

In [2]:
climate = Climate()
soil = Soil(texture='loam')
crop = Crop(soil=soil)
soil.set_nZr(crop)

225.5

In [3]:
model = CropModel(crop=crop, climate=climate, soil=soil)
model.run()

done


In [4]:
model.output()

,kc,LAI,stress,R,s,E,ET,T,L,dsdt
0,0.300000,0.750000,1.000000,0.000000,0.300000,0.099097,0.099097,0.000000,0.0,-0.099097
1,0.300000,0.750000,1.000000,0.000000,0.299561,0.098010,0.098010,0.000000,0.0,-0.098010
2,0.300000,0.750000,1.000000,0.000000,0.299126,0.096939,0.096939,0.000000,0.0,-0.096939
3,0.300000,0.750000,1.000000,0.454231,0.298696,0.095884,0.095884,0.000000,0.0,0.358347
4,0.300000,0.750000,1.000000,0.000000,0.300285,0.099804,0.099804,0.000000,0.0,-0.099804
...,...,...,...,...,...,...,...,...,...,...
175,0.666667,1.666667,0.090763,0.000000,0.546607,0.723858,2.587142,1.863284,0.0,-2.587142
176,0.653333,1.633333,0.114432,0.982355,0.535134,0.695101,2.424401,1.729300,0.0,-1.442046
177,0.640000,1.600000,0.128814,0.000000,0.528739,0.683936,2.325135,1.641199,0.0,-2.325135
178,0.626667,1.566667,0.153796,1.247051,0.518428,0.658514,2.182146,1.523632,0.0,-0.935095
